<a href="https://colab.research.google.com/github/ch23s020/Assignment3/blob/main/Assignment3_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import gdown
import random
import time
import pandas as pd
import csv


In [ ]:

# Define the download URLs for Google Sheets as CSV




Data laod and  Pre-Processing

In [3]:
# Download dataset from Google Drive

train_url = "https://docs.google.com/spreadsheets/d/11duz5Vbqay5TVn_uyglVQVcEZllTbWQt_8zTt2TcBSA/export?format=csv"

valid_url = "https://docs.google.com/spreadsheets/d/1KbKFfxFkMddkZde0r5PWKnQ0vzdh-XihxsMP7XUFDJc/export?format=csv"

test_url = "https://docs.google.com/spreadsheets/d/1ItKDweGPNtzWiF3rs0jzKjh7ZRRkas2hz7yWvbt4yzQ/export?format=csv"

# Paths to save the files

train_output = '/content/train_data.csv'

valid_output = '/content/valid_data.csv'

test_output = '/content/test_data.csv'

# Download the files

gdown.download(train_url, train_output, quiet=False)

gdown.download(valid_url, valid_output, quiet=False)

gdown.download(test_url, test_output, quiet=False)


# Function to load data

def load_data(file_path):

    data = []

    with open(file_path, 'r', encoding='utf-8') as csvfile:

        csvreader = csv.reader(csvfile)

        for idx, row in enumerate(csvreader):

            if row:  # Check if the row is not empty

                data.append(row)

    return data

# Load the data

train_data = load_data(train_output)

valid_data = load_data(valid_output)

test_data = load_data(test_output)

# Print a sample from each dataset to verify

print("Sample from train data:", train_data[0])

print("Sample from valid data:", valid_data[0])

print("Sample from test data:", test_data[0])




# Separated the Training, Validation and Test Data files for Marathi English Transliteration.

# def load_data(file_path):

#     data = []

#     with open(file_path, 'r', encoding='utf-8') as csvfile:

#         csvreader = csv.reader(csvfile)

#         # Trying to remove the Index error while passing the separated csv file.
#         for idx, row in enumerate(csvreader):

#             try:
#                 x = str(row[0])

#                 y = str(row[1])

#                 data.append((x, y))

#             except IndexError:   # To locate the issue

#                 print(f"IndexError in row {idx + 1}: {row}")
#     return data

# data = load_data(output)



# Data Preparation

# class TransliterationDataset(Dataset):

#     def __init__(self, data, char2index, max_length=20):

#         self.data = data

#         self.max_length = max_length

#         self.char2index = char2index



#     def __len__(self):

#         return len(self.data)

#     def __getitem__(self, idx):

#         x, y = self.data[idx]

#         x_indices = [self.char2index[c] for c in x] + [self.char2index['<PAD>']] * (self.max_length - len(x))

#         y_indices = [self.char2index[c] for c in y] + [self.char2index['<PAD>']] * (self.max_length - len(y))

#         return torch.tensor(x_indices), torch.tensor(y_indices), len(x), len(y)

# def collate_fn(batch):

#     x, y, x_lengths, y_lengths = zip(*batch)

#     x = torch.nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=char2index['<PAD>'])

#     y = torch.nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=char2index['<PAD>'])

#     return x, y


/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://docs.google.com/spreadsheets/d/11duz5Vbqay5TVn_uyglVQVcEZllTbWQt_8zTt2TcBSA/export?format=csv
To: /content/train_data.csv
2.23MB [00:00, 5.76MB/s]
Downloading...
From: https://docs.google.com/spreadsheets/d/1KbKFfxFkMddkZde0r5PWKnQ0vzdh-XihxsMP7XUFDJc/export?format=csv
To: /content/valid_data.csv
143kB [00:00, 4.52MB/s]
Downloading...
From: https://docs.google.com/spreadsheets/d/1ItKDweGPNtzWiF3rs0jzKjh7ZRRkas2hz7yWvbt4yzQ/export?format=csv
To: /content/test_data.csv
149kB [00:00, 4.11MB/s]


Sample from train data: ['fusharun', 'फुशारुन']
Sample from valid data: ['garvyabarobarach', 'गारव्याबरोबरच']
Sample from test data: ['heetler', 'हिटलर']


RNN


In [ ]:
class EncoderRNN(pass):

class DecoderRNN(pass):

class Seq2Seq(nn.Module):



Training
